In [28]:
from transformers import GPT2TokenizerFast, GPT2Tokenizer
from copy import deepcopy
import numpy as np
from random import random
from time import time

In [29]:

from embedding_mask import sum_masks


In [30]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
vocabi = {v:k for k, v in vocab.items()}
SRC = "asd_pol.txt"
with open(SRC, "r") as f:
    all_txt = f.read()
def get_imput(txt, length=512):
    bi = int(random()*(len(txt)-1000))
    return txt[bi:bi+length]

In [31]:

from functools import cache


def tokenize_atoms(txt, tokenizer):
    atxt = "".join(tokenizer.tokenize(txt))
    atxt = list(atxt)
    eatxt = tokenizer.encode(atxt)
    return eatxt

c = 0

def lazy_past_tokens_gather(atokens, tokenizer): #dev no dobra, może tak być dla tokenów kłopotliwych, ale dla reszty już nie powinno
    global c #dev
    c+=1 #dev
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        past_emb_pos.append(atokens[:i])
    return past_emb_pos

def past_tokens_gather(atokens, tokenizer):
    past_emb_pos = []
    for i in range(1, len(atokens)+1):
        pdec = tokenizer.decode(atokens[:i])
        if '�' in pdec:
            return lazy_past_tokens_gather(atokens, tokenizer)
        penc = tokenizer.encode(pdec)
        past_emb_pos.append(penc)
    return past_emb_pos

def create_emb_pos_mask(emb_pos):
    res = []
    # mask = np.eye(len(emb_pos)).astype(bool)
    mask = np.eye(len(emb_pos))
    for embt, m in zip(emb_pos, mask):
        res.append(embt[-1])
        for ie in embt[:-1]:
            is_added = True if res[:-1] == [] else False
            for j, je in enumerate(res[:-1]):
                if ie == je:
                    m[j] = 1
                    is_added = True
                    break
            if not is_added:
                raise Exception("target len != input len")
    return np.array(res)[:,0], np.array(res)[:,1], mask

# @cache
def tokenizex_encode_full(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m


In [32]:
def tokenizex_encode(txt, tokenizer, atomizer):
    tokens = tokenizer.encode(txt)

    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    for t in tokens:
        emb, pos, mask = atomizer[t]
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [33]:
import re


def remove_whitespace_before_dots_p(text):
    # Używamy wyrażenia regularnego, aby zastąpić białe znaki przed kropkami
    modified_text = re.sub(r'\s+\.', '.', text)
    modified_text = re.sub(r'\n+\n', '\n', modified_text)
    return modified_text

In [34]:
def validate(txt, emb, pos, mask, tokenizer):
    txt = remove_whitespace_before_dots_p(txt)
    dec = tokenizer.decode(emb[mask[-1].astype(bool)])
    assert dec == txt

    lemb = tokenize_atoms(txt, tokenizer)
    assert len(lemb) == len(emb)

    comp = pos[mask[-1].astype(bool)]
    for a, b in zip(comp.astype(int), list(range(len(comp)))):
        assert a == b

In [35]:
def tokenizex_w_encode(txt, tokenizer):
    res_emb = []
    res_pos = []
    partial_masks = []
    pos_len = 0
    
    txtw = [" " + e for e in txt.split(" ")]
    if len(txtw[0]) == 1:
        del txtw[0]
    else:
        txtw[0] = txtw[0][1:]
    
    for w in txtw:
        emb, pos, mask = tokenizex_encode_full(w, tokenizer)
        res_emb.extend(emb)
        res_pos.extend(pos+pos_len)
        pos_len += pos[-1] + 1
        partial_masks.append(mask)   
    full_mask = sum_masks(len(res_emb), partial_masks)
    return np.array(res_emb), np.array(res_pos), full_mask


In [36]:
txt = " return np.array(res_emb), np.array(res_pos), full_mask"
txt = """  𡨸 𡨸żóąs"""
txt = """cial Maurice Rossel reported that conditions there were "almost normal".
2014 – Under the terms of UN Security C"""
txt = tokenizer.prepare_for_tokenization(txt)[0]
txt = remove_whitespace_before_dots_p(txt)
txt

'cial Maurice Rossel reported that conditions there were "almost normal".\n2014 – Under the terms of UN Security C'

In [37]:
# txt = errorous
txt = """\n\nY"""
txt = '\n\nf\ngh\n\n\n\nSta'

In [38]:
re.sub(r'\n+\n', '\n', txt)

'\nf\ngh\nSta'

In [39]:
txt = remove_whitespace_before_dots_p(txt)

In [40]:
tenc = tokenize_atoms(txt, tokenizer)
# print(tenc)
res = past_tokens_gather(tenc, tokenizer)
# print(res)
g = [list(zip(e, np.arange(len(e)))) for e in res]
e, p, m = create_emb_pos_mask(g)
# print(e)
# print(m)
validate(txt, e, p, m, tokenizer)

In [41]:
e, p, m = tokenizex_encode_full(txt, tokenizer)
validate(txt, e, p, m, tokenizer)

In [42]:
ee, pp, mm = tokenizex_w_encode(txt, tokenizer)
# print(ee)
# print(mm)
validate(txt, ee, pp, mm, tokenizer)

In [43]:
def tokenizer_def(txt, tokenizer):
    return tokenizer.encode(txt), None, None

def tokenizer_my1(txt, tokenizer):
    tenc = tokenize_atoms(txt, tokenizer)
    res = past_tokens_gather(tenc, tokenizer)
    g = [list(zip(e, np.arange(len(e)))) for e in res]
    e, p, m = create_emb_pos_mask(g)
    return e, p, m

def tokenizer_my2(txt, tokenizer):
    e, p, m= tokenizex_w_encode(txt, tokenizer)
    return e, p, m

In [44]:
errorous = ""
def time_valid_tokenization(txt_data, tokenizer, def_tokenizer, C, lbl_val = False):
    st = 0
    global errorous
    for i in range(C):
        txt = get_imput(txt_data, 512)
        # txti = def_tokenizer.decode(def_tokenizer.encode(get_imput(txt_data, 112)))
        txt = def_tokenizer.prepare_for_tokenization(txt)[0]
        txt = remove_whitespace_before_dots_p(txt)
        t1 = time()
        errorous = txt
        emb, pos, mask = tokenizer(txt, def_tokenizer)
        st += time()-t1

        if lbl_val: 
            validate(txt, emb, pos, mask, def_tokenizer)

    return st


In [45]:
txt = " ż ż ... asda ... "
txt = remove_whitespace_before_dots_p(txt)
emb, pos, mask = tokenizer_my2(txt, tokenizer)
validate(txt, emb, pos, mask, tokenizer)

In [46]:
default = time_valid_tokenization(all_txt, tokenizer_def, tokenizer, 100, False)

In [47]:
my2 = time_valid_tokenization(all_txt, tokenizer_my2, tokenizer, 100, True)

In [48]:
# my1 = time_valid_tokenization(all_txt, tokenizer_my1, tokenizer, 500, True)

In [49]:
llm_examples = 128*500000

# print(f"my1/default ratio {my1/default}")
print(f"my2/default ratio {my2/default}")


my2/default ratio 59.47179786923138


In [50]:
from research.tokenizex.model.tokenizer import TokenizexTokenizer
tokenizex = TokenizexTokenizer()

In [51]:
txt = " ż ż ... asda ... "

e, p, m = tokenizex.text_to_ids_pos_mask(txt)

validate(txt, e, p, m, tokenizer)

In [52]:
def tokenizex_fin(txt, tokenizex:TokenizexTokenizer):
    e,p,m = tokenizex.text_to_ids_pos_mask(txt)
    tokenizex.validate_tokenization(txt,e,p,m)

In [53]:
tokenizex_fin("sdfsdfsd", tokenizex)

In [54]:
t3 = "example text to tokenize and, żargon ołówka źle wpływa detokenize, ślóza."
t3 = "example."
e2  = np.array([69])
p2  = np.array([0])
m2  = np.array([1])
t2 = tokenizer.decode(e2)

e1, p1, m1 = encode_em(t1, tokenizer)
e3, p3, m3 = encode_em(t3, tokenizer)
txt = t1+t2+t3
txt

emb = np.concatenate([e1,e2,e3])
pos = sum_pos([p1,p2,p3])
mlen = m1.shape[0]+m2.shape[0]+m3.shape[0]
mask = sum_masks(mlen, [m1, m2, m3])


NameError: name 'encode_em' is not defined